### Step1 - 导入相关包 & 初始化设置

In [1]:
import os, sys

from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 默读取当前目录下的 .env 文件, 可以通过 dotenv_path 来修改
from dotenv import load_dotenv
root_dir = os.path.dirname(os.getcwd())
load_dotenv(dotenv_path=os.path.join(root_dir, ".env"))

# 记录日志
from loguru import logger
logger.remove()
logger.add(sys.stderr, level=os.getenv("LOG_LEVEL"))

1

### Step2 - 加载 嵌入模型 & 向量数据库

In [2]:
# 加载嵌入模型
embed_model = OllamaEmbeddings(
    model=os.getenv("OLLAMA_EMB_MODEL"),
    base_url=os.getenv("OLLAMA_BASE_URL"),
)
embed_model

OllamaEmbeddings(model='qwen3-embedding:4b', validate_model_on_init=False, base_url='http://localhost:11434', client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [3]:
# 加载向量数据库
vector_db = Chroma(
    collection_name="example_collection",
    embedding_function=embed_model,
    persist_directory="./chroma_db",
)
vector_db

### Step3 - 进行语义检索

In [5]:
# 方法1: 使用文本相似度搜索
results = vector_db.similarity_search("注意力机制", k=2)
for doc in results:
    logger.info(f"method1 - doc content: {doc.page_content[:20]}")

2025-12-07 14:36:13.757 | INFO     | __main__:<module>:4 - method1 - doc content: Attention Is All You
2025-12-07 14:36:13.758 | INFO     | __main__:<module>:4 - method1 - doc content: Attention Visualizat


In [ ]:
# 方法2: 使用嵌入向量来查询 (本质和方法1相同)
query_vector = embed_model.embed_query("注意力机制")
results = vector_db.similarity_search_by_vector(query_vector, k=2)
for doc in results:
    logger.info(f"method2 - doc content: {doc.page_content[:20]}")

2025-12-07 14:36:42.871 | INFO     | __main__:<module>:5 - method2 - doc content: Attention Is All You
2025-12-07 14:36:42.872 | INFO     | __main__:<module>:5 - method2 - doc content: Attention Visualizat


In [7]:
# 方法3: 使用文本相似度搜索 (带分数)
results = vector_db.similarity_search_with_score("注意力机制", k=2)
for doc, score in results:
    # round 方法为保留多少位小数
    logger.info(f"method3 - score: {round(score, 4)} | doc content: {doc.page_content[:20]}")

2025-12-07 14:37:33.453 | INFO     | __main__:<module>:5 - method3 - score: 0.7793 | doc content: Attention Is All You
2025-12-07 14:37:33.454 | INFO     | __main__:<module>:5 - method3 - score: 0.8309 | doc content: Attention Visualizat
